<a href="https://colab.research.google.com/github/kmalhotra18/Code-Comment-Generator/blob/main/Code_Comment_Generator_GPT_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q OpenAI
!pip install -q google-generativeai
!pip install -q python-dotenv
!pip install -q anthropic
!pip install -q gradio

In [ ]:
# Sign in to OpenAI using Secrets in Colab

import os
from openai import OpenAI

# Get the API key from the environment variable
openai_api_key = os.getenv('OPENAI_API_KEY')
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# If the API key is not found in the environment variable,
# try getting it from a .env file (if you're using one)
if openai_api_key is None:
    from dotenv import load_dotenv
    load_dotenv()
    openai_api_key = os.getenv('OPENAI_API_KEY')

# If the API key is still None, raise an error
if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable not set. Please set it or create a .env file with the key.")

if not GEMINI_API_KEY:
    raise ValueError("⚠️ Gemini API key not loaded. Please check your .env or environment setup.")

# Initialize the OpenAI client with the API key
openai = OpenAI(api_key=openai_api_key)

In [ ]:
from openai import OpenAI
import google.generativeai as genai
import os # Import the os module

# Set up clients
openai_client = OpenAI(api_key=os.getenv('OPENAI_API_KEY')) # Use os.getenv to get the API key
genai.configure(api_key=os.getenv('GEMINI_API_KEY')) # Use os.getenv to get the API key
genai.configure(api_key=GEMINI_API_KEY)

def generate_with_openai(code):
    prompt = (
        "You're a helpful assistant that comments Python code. "
        "Given any raw Python code, return the same code with meaningful docstrings and inline comments. "
        "Preserve formatting. Only return the modified code."
    )

    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": code}
        ],
        temperature=0.3
    )

    return response.choices[0].message.content


def generate_with_gemini(code):
    model = genai.GenerativeModel("gemini-2.0-flash")

    prompt = (
        "You're a helpful assistant that comments Python code. "
        "Given the following code, add meaningful docstrings and inline comments. "
        "Return the improved code with formatting preserved:\n\n" + code
    )

    response = model.generate_content(prompt)
    return response.text.strip()

In [ ]:
def generate_comments(code, model_choice):
    if not code.strip():
        return "⚠️ Please paste code or upload a file first."

    if model_choice == "OpenAI (GPT-4o)":
        return generate_with_openai(code)
    elif model_choice == "Gemini (Google)":
        return generate_with_gemini(code)
    else:
        return "❌ Invalid model selected."


In [ ]:
import gradio as gr

with gr.Blocks(title="AI Code Comment Generator") as app:
    gr.Markdown("## 🧠 Add Docstrings & Inline Comments with GPT or Gemini")

    model_dropdown = gr.Dropdown(
        choices=["OpenAI (GPT-4o)", "Gemini (Google)"],
        label="Select LLM",
        value="OpenAI (GPT-4o)"
    )

    with gr.Tab("📋 Paste Code"):
        code_input = gr.Code(label="Paste Python Code", language="python")

    with gr.Tab("📁 Upload File"):
        file_input = gr.File(label="Upload a .py File", file_types=[".py"])

    output = gr.Code(label="📄 Commented Code", language="python")
    run_button = gr.Button("✨ Generate Comments")

    def process_input(code, file, model_choice):
        if file is not None and not code:
            code = file.read().decode("utf-8")
        return generate_comments(code, model_choice)

    run_button.click(
        fn=process_input,
        inputs=[code_input, file_input, model_dropdown],
        outputs=output
    )

app.launch()


In [ ]:
import google.generativeai as genai
genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel("gemini-pro")
response = model.generate_content("Add comments to:\n\ndef add(x, y): return x + y")
print(response.text)